<a href="https://colab.research.google.com/github/jarrodmhicks/neuromatch_similarity/blob/functionality_updates/notebooks/getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Access neuromatch_similarity code
# uncomment lines below

## if running in colab:
# !git clone https://github.com/jarrodmhicks/neuromatch_similarity.git
## then switch to specific branch
# !cd neuromatch_similarity && git checkout functionality_updates


## if running locally:
# import sys
## point to where repo is saved locally
# sys.path.append('/om2/user/jmhicks/projects/NeuroMatchSimilarity/code/')

Mounted at /content/drive
Cloning into 'neuromatch_similarity'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 58 (delta 24), reused 40 (delta 13), pack-reused 0
Unpacking objects: 100% (58/58), 30.12 KiB | 2.32 MiB/s, done.


In [2]:
#@title Import utils
import neuromatch_similarity.utils as nms
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Subset

In [3]:
#@title Test loading data
# set device
device = nms.helpers.set_device()
print(device)

# load human data
dataset = nms.datasets.SimilarityDataset(device=device)

# load network features
network_features = nms.datasets.NetworkFeatures(model_name='alexnet',
                                                layer_name='classifier.5',
                                                device=device)

Current device: cuda
cuda


In [4]:
#@title Test model setup
transform = nms.transforms.LinearDiagonal(network_features.shape[1])
distance = nms.distances.Euclidean() 
model = nms.model.DistanceModel(transform, distance).to(device)

In [5]:
#@title Test training
# setup dataloader
# train with first 10k trials for testing
n_train = 10000
batch_size = 1000
train_indices = torch.arange(n_train)
train_dataset = Subset(dataset, train_indices)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# initialize loss function and optimizer
learning_rate = 0.001
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# run training loop
num_epochs = 100
summary_every = 10
train_losses = nms.model.train(network_features, model, optimizer, 
                               loss_function, train_loader, num_epochs, 
                               summary_every)

Epoch [10/100]
	Step [10/10], Loss: 1.055641
Epoch [20/100]
	Step [10/10], Loss: 1.020745
Epoch [30/100]
	Step [10/10], Loss: 1.028562
Epoch [40/100]
	Step [10/10], Loss: 1.018771
Epoch [50/100]
	Step [10/10], Loss: 1.009783
Epoch [60/100]
	Step [10/10], Loss: 1.011662
Epoch [70/100]
	Step [10/10], Loss: 1.007672
Epoch [80/100]
	Step [10/10], Loss: 1.007606
Epoch [90/100]
	Step [10/10], Loss: 1.013256
Epoch [100/100]
	Step [10/10], Loss: 1.010323
